<a href="https://colab.research.google.com/github/socome/ML_study/blob/master/Lec05_2_exercise_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/socome/ML_study.git

Cloning into 'ML_study'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 58 (delta 22), reused 19 (delta 3), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [0]:
ls ML_study

 data_01_test_score.csv   Lec04_1_3.ipynb         'Lec05-2(realdatatest).ipynb'
 data-03-diabetes.csv     Lec04_1_4.ipynb          Lec05.ipynb
 Lec04_1_1.ipynb          lec05_2_exercise.ipynb   Lec6.ipynb
 Lec04_1_2.ipynb          Lec05-2-exercise.ipynb


In [1]:
import tensorflow as tf
import numpy as np

numpy_xy = np.loadtxt('ML_study/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = numpy_xy[:, 0:-1]
y_data = numpy_xy[:, [-1]]

filename_queue = tf.train.string_input_producer(['ML_study/data-03-diabetes.csv'], shuffle=False, name='filename_queue')
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
record_defaults = [[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=30)

#placeholder for a tensor that will be always feed
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([8,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

#Hypothesis using sigmoid
#if not using sigmoid (1., 1+tf.exp(tf.matmul(X,W) + b))
hypothesis = tf.sigmoid(tf.matmul(X,W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y*tf.log(hypothesis)+(1-Y)*tf.log(1-hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)


#cast => true(1)/false(0) 
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y), dtype=tf.float32))

# parameters
training_epochs = 20

#Launch graph
with  tf.Session() as sess:

    #initialize
    sess.run(tf.global_variables_initializer())

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for epoch in range(training_epochs) :
      avg_cost = 0
      
      for i in range(30):
          x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
          cost_val, _ = sess.run([cost,train], feed_dict={X:x_batch, Y:y_batch})  
          avg_cost += cost_val / 30
          print(i ,cost_val) 
            
    coord.request_stop()
    coord.join(threads)
    
    #Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\n Hypothesis : ", h, "\nCorrect (Y) :", c, "\nAccuracy:", a)

OSError: ignored